In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
#detector

from keras.models import *
from keras.layers import *
import tensorflow as tf

class L2Norm(Layer):
    '''
    Code borrows from https://github.com/flyyufelix/cnn_finetune
    '''
    def __init__(self, weights=None, axis=-1, gamma_init='zero', n_channels=256, scale=10, **kwargs):
        self.axis = axis
        self.gamma_init = tf.keras.initializers.get(gamma_init)
        self.initial_weights = weights
        self.n_channels = n_channels
        self.scale = scale
        super(L2Norm, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        self.gamma = K.variable(self.gamma_init((self.n_channels,)), name='{}_gamma'.format(self.name))
        self.trainable_weights1 = [self.gamma]
        self.built = True

    def call(self, x, mask=None):
        norm = K.sqrt(K.sum(K.square(x), axis=-1, keepdims=True)) + K.epsilon()
        x = x / norm * self.gamma
        return x

    def get_config(self):
        config = {"axis": self.axis}
        base_config = super(L2Norm, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

def s3fd_keras():    
    inp = Input((None,None,3))
    
    conv1_1 = Conv2D(filters=64, kernel_size=3, padding="same", name="conv1_1", activation="relu")(inp)
    conv1_2 = Conv2D(filters=64, kernel_size=3, padding="same", name="conv1_2", activation="relu")(conv1_1)
    maxpool1 = MaxPooling2D()(conv1_2)
    
    conv2_1 = Conv2D(filters=128, kernel_size=3, padding="same", name="conv2_1", activation="relu")(maxpool1)
    conv2_2 = Conv2D(filters=128, kernel_size=3, padding="same", name="conv2_2", activation="relu")(conv2_1)
    maxpool2 = MaxPooling2D()(conv2_2)
    
    conv3_1 = Conv2D(filters=256, kernel_size=3, padding="same", name="conv3_1", activation="relu")(maxpool2)
    conv3_2 = Conv2D(filters=256, kernel_size=3, padding="same", name="conv3_2", activation="relu")(conv3_1)
    conv3_3 = Conv2D(filters=256, kernel_size=3, padding="same", name="conv3_3", activation="relu")(conv3_2)
    f3_3 = conv3_3
    maxpool3 = MaxPooling2D()(conv3_3)
    
    conv4_1 = Conv2D(filters=512, kernel_size=3, padding="same", name="conv4_1", activation="relu")(maxpool3)
    conv4_2 = Conv2D(filters=512, kernel_size=3, padding="same", name="conv4_2", activation="relu")(conv4_1)
    conv4_3 = Conv2D(filters=512, kernel_size=3, padding="same", name="conv4_3", activation="relu")(conv4_2)
    f4_3 = conv4_3
    maxpool4 = MaxPooling2D()(conv4_3)
    
    conv5_1 = Conv2D(filters=512, kernel_size=3, padding="same", name="conv5_1", activation="relu")(maxpool4)
    conv5_2 = Conv2D(filters=512, kernel_size=3, padding="same", name="conv5_2", activation="relu")(conv5_1)
    conv5_3 = Conv2D(filters=512, kernel_size=3, padding="same", name="conv5_3", activation="relu")(conv5_2)
    f5_3 = conv5_3
    maxpool5 = MaxPooling2D()(conv5_3)
    
    
    # ========== Note ==========
    # Be careful about the zeropadding difference when strides >= 2
    fc6 = ZeroPadding2D(3)(maxpool5)
    fc6 = Conv2D(filters=1024, kernel_size=3, name="fc6", activation="relu")(fc6)
    fc7 = Conv2D(filters=1024, kernel_size=1, name="fc7", activation="relu")(fc6)
    ffc7 = fc7
    conv6_1 = Conv2D(filters=256, kernel_size=1, name="conv6_1", activation="relu")(fc7)
    f6_1 = conv6_1
    conv6_2 = ZeroPadding2D()(conv6_1)
    conv6_2 = Conv2D(filters=512, kernel_size=3, strides=2, name="conv6_2", activation="relu")(conv6_2)
    f6_2 = conv6_2
    conv7_1 = Conv2D(filters=128, kernel_size=1, name="conv7_1", activation="relu")(f6_2)
    f7_1 = conv7_1
    conv7_2 = ZeroPadding2D()(conv7_1)
    conv7_2 = Conv2D(filters=256, kernel_size=3, strides=2, name="conv7_2", activation="relu")(conv7_2)
    f7_2 = conv7_2
    
    f3_3 = L2Norm(n_channels=256, scale=10, name="conv3_3_norm")(f3_3)
    f4_3 = L2Norm(n_channels=512, scale=8, name="conv4_3_norm")(f4_3)
    f5_3 = L2Norm(n_channels=512, scale=5, name="conv5_3_norm")(f5_3)
    
    cls1 = Conv2D(filters=4, kernel_size=3, padding="same", name="conv3_3_norm_mbox_conf")(f3_3)
    reg1 = Conv2D(filters=4, kernel_size=3, padding="same", name="conv3_3_norm_mbox_loc")(f3_3)
    cls2 = Conv2D(filters=2, kernel_size=3, padding="same", name="conv4_3_norm_mbox_conf")(f4_3)
    reg2 = Conv2D(filters=4, kernel_size=3, padding="same", name="conv4_3_norm_mbox_loc")(f4_3)
    cls3 = Conv2D(filters=2, kernel_size=3, padding="same", name="conv5_3_norm_mbox_conf")(f5_3)
    reg3 = Conv2D(filters=4, kernel_size=3, padding="same", name="conv5_3_norm_mbox_loc")(f5_3)
    cls4 = Conv2D(filters=2, kernel_size=3, padding="same", name="fc7_mbox_conf")(ffc7)
    reg4 = Conv2D(filters=4, kernel_size=3, padding="same", name="fc7_mbox_loc")(ffc7)
    
    cls5 = Conv2D(filters=2, kernel_size=3, padding="same", name="conv6_2_mbox_conf")(f6_2)
    reg5 = Conv2D(filters=4, kernel_size=3, padding="same", name="conv6_2_mbox_loc")(f6_2)
    cls6 = Conv2D(filters=2, kernel_size=3, padding="same", name="conv7_2_mbox_conf")(f7_2)
    reg6 = Conv2D(filters=4, kernel_size=3, padding="same", name="conv7_2_mbox_loc")(f7_2)
    
    def get_chunk(x, c):
        return tf.split(x, c, axis=-1)
    chunk = Lambda(lambda x: get_chunk(x, 4))(cls1)
    bmax = Lambda(lambda chunk: K.maximum(K.maximum(chunk[0], chunk[1]), chunk[2]))(chunk)
    cls1 = Concatenate(axis=-1)([bmax, chunk[3]])
    
    return Model(inp, [cls1, reg1, cls2, reg2, cls3, reg3, cls4, reg4, cls5, reg5, cls6, reg6])

In [54]:
import numpy as np
from scipy import special
#from .model import s3fd_keras

class S3FD():
    def __init__(self, weights_path="/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/s3fd_keras_weights.h5"):
        self.net = s3fd_keras()
        self.net.load_weights(weights_path)
    
    def detect_face(self, image):
        bboxlist = self.detect(self.net, image)
        keep = self.nms(bboxlist, 0.3)
        bboxlist = bboxlist[keep, :]
        bboxlist = [x for x in bboxlist if x[-1] > 0.5]
        return bboxlist
    
    def detect(self, net, img):    
        def softmax(x, axis=-1):
            return np.exp(x - special.logsumexp(x, axis=axis, keepdims=True))
        img = img - np.array([104, 117, 123])
        if img.ndim == 3:
            img = img[np.newaxis, ...]
        elif img.ndim == 5:
            img = np.squeeze(img)

        BB, HH, WW, CC = img.shape
        olist = net.predict(img) # output a list of 12 predicitons in different resolution

        bboxlist = []
        for i in range(len(olist) // 2):
            olist[i * 2] = softmax(olist[i * 2], axis=-1)
        olist = [oelem for oelem in olist]
        for i in range(len(olist) // 2):
            ocls, oreg = olist[i * 2], olist[i * 2 + 1]
            FB, FH, FW, FC = ocls.shape  # feature map size
            stride = 2**(i + 2)    # 4,8,16,32,64,128
            anchor = stride * 4
            poss = zip(*np.where(ocls[:, :, :, 1] > 0.05))
            for Iindex, hindex, windex in poss:
                axc, ayc = stride / 2 + windex * stride, stride / 2 + hindex * stride
                score = ocls[0:1, hindex, windex, 1]
                loc = oreg[0:1, hindex, windex, :]
                priors = np.array([[axc / 1.0, ayc / 1.0, stride * 4 / 1.0, stride * 4 / 1.0]])
                variances = [0.1, 0.2]
                box = self.decode(loc, priors, variances)
                x1, y1, x2, y2 = box[0] * 1.0
                bboxlist.append([x1, y1, x2, y2, score])
        bboxlist = np.array(bboxlist)
        if 0 == len(bboxlist):
            bboxlist = np.zeros((1, 5))            
        return bboxlist

    @staticmethod
    def decode(loc, priors, variances):
        """Decode locations from predictions using priors to undo
        the encoding we did for offset regression at train time.
        Args:
            loc (tensor): location predictions for loc layers,
                Shape: [num_priors,4]
            priors (tensor): Prior boxes in center-offset form.
                Shape: [num_priors,4].
            variances: (list[float]) Variances of priorboxes
        Return:
            decoded bounding box predictions
        """
        boxes = np.concatenate((
            priors[:, :2] + loc[:, :2] * variances[0] * priors[:, 2:],
            priors[:, 2:] * np.exp(loc[:, 2:] * variances[1])), 1)
        boxes[:, :2] -= boxes[:, 2:] / 2
        boxes[:, 2:] += boxes[:, :2]
        return boxes

    @staticmethod
    def nms(dets, thresh):
        if 0 == len(dets):
            return []
        x1, y1, x2, y2, scores = dets[:, 0], dets[:, 1], dets[:, 2], dets[:, 3], dets[:, 4]
        areas = (x2 - x1 + 1) * (y2 - y1 + 1)
        order = scores.argsort()[::-1]

        keep = []
        while order.size > 0:
            i = order[0]
            keep.append(i)
            xx1, yy1 = np.maximum(x1[i], x1[order[1:]]), np.maximum(y1[i], y1[order[1:]])
            xx2, yy2 = np.minimum(x2[i], x2[order[1:]]), np.minimum(y2[i], y2[order[1:]])
            w, h = np.maximum(0.0, xx2 - xx1 + 1), np.maximum(0.0, yy2 - yy1 + 1)
            ovr = w * h / (areas[i] + areas[order[1:]] - w * h)
            inds = np.where(ovr <= thresh)[0]
            order = order[inds + 1]
        return keep

In [55]:
import cv2
import numpy as np
from pathlib import Path
import tensorflow as tf
from keras import backend as K

#from .s3fd.s3fd_detector import S3FD
#from .mtcnn.mtcnn_detector import MTCNN
#from .landmarks_detector import FANLandmarksDetector

#FILE_PATH = Path(__file__).parent.resolve()

class BaseFaceDetector():
    def __init__(self):
        pass
    
    def detect_face(self):
        raise NotImplementedError

'''class MTCNNFaceDetector(BaseFaceDetector):
    def __init__(self, weights_path=FILE_PATH / "mtcnn"):
        self.face_detector = MTCNN(weights_path)
        
    def detect_face(self, image):
        # Output bbox coordinate has ordering (y0, x0, y1, x1)
        return self.face_detector.detect_face(image)
        
    def batch_detect_face(self, image):
        raise NotImplementedError'''

class S3FaceDetector(BaseFaceDetector):
    def __init__(self, weights_path="/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/s3fd_keras_weights.h5"):
        self.face_detector = S3FD(weights_path)
        
    def detect_face(self, image):
        # Output bbox coordinate has ordering (y0, x0, y1, x1)
        return self.face_detector.detect_face(image)
        
    def batch_detect_face(self, image):
        raise NotImplementedError



class FaceAlignmentDetector(BaseFaceDetector):
    def __init__(self, 
                 fd_weights_path="/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/s3fd_keras_weights.h5", 
                 lmd_weights_path="/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/2DFAN-4_keras.h5",
                 fd_type="s3fd"):
        self.fd_type = fd_type.lower()
        if fd_type.lower() == "s3fd":
            self.fd = S3FaceDetector(fd_weights_path)
        elif fd_type.lower() == "mtcnn":
            self.fd = MTCNNFaceDetector()
        else:
            raise ValueError(f"Unknown face detector {face_detector}.")
        
        self.lmd_weights_path = lmd_weights_path
        self.lmd = None
    
    def build_FAN(self):
        self.lmd = FANLandmarksDetector(self.lmd_weights_path)

    def preprocess_s3fd_bbox(self,bbox_list):
        # Convert coord (y0, x0, y1, x1) to (x0, y0, x1, y1)
        return [np.array([bbox[1], bbox[0], bbox[3], bbox[2], bbox[4]]) for bbox in bbox_list]
    
    def detect_face(self, image, with_landmarks=True):
        """
        Returns: 
            bbox_list: bboxes in [x0, y0, x1, y1] ordering (x is the vertical axis, y the height).
            landmarks_list: landmark points having shape (68, 2) with ordering [[x0, y0], [x1, y1], ..., [x67, y67].
        """
        if self.fd_type == "s3fd":
            bbox_list = self.fd.detect_face(image)
        elif self.fd_type == "mtcnn":
            bbox_list = self.fd.detect_face(image)
        if len(bbox_list) == 0:
            return [], []
            
        if with_landmarks:
            if self.lmd == None:
                print("Building FAN for landmarks detection...")
                self.build_FAN()
                print("Done.")
            landmarks_list = []
            for bbox in bbox_list:
                pnts = self.lmd.detect_landmarks(image, bounding_box=bbox)[-1]
                landmarks_list.append(np.array(pnts))
            landmarks_list = [self.post_process_landmarks(landmarks) for landmarks in landmarks_list]
            bbox_list = self.preprocess_s3fd_bbox(bbox_list)
            return bbox_list, landmarks_list
        else:
            bbox_list = self.preprocess_s3fd_bbox(bbox_list)
            return bbox_list
    
    def batch_detect_face(self, images, **kwargs):
        raise NotImplementedError


In [56]:
#for getting bboxes we have fd
fd = FaceAlignmentDetector(lmd_weights_path="/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/s3fd_keras_weights.h5")

In [57]:
#verification

from functools import partial

from keras.models import Model
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Concatenate
from keras.layers import Conv2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input
from keras.layers import Lambda
from keras.layers import MaxPooling2D
from keras.layers import add
from keras import backend as K


def scaling(x, scale):
    return x * scale


def conv2d_bn(x,
              filters,
              kernel_size,
              strides=1,
              padding='same',
              activation='relu',
              use_bias=False,
              name=None):
    x = Conv2D(filters,
               kernel_size,
               strides=strides,
               padding=padding,
               use_bias=use_bias,
               name=name)(x)
    if not use_bias:
        bn_axis = 1 if K.image_data_format() == 'channels_first' else 3
        bn_name = _generate_layer_name('BatchNorm', prefix=name)
        x = BatchNormalization(axis=bn_axis, momentum=0.995, epsilon=0.001,
                               scale=False, name=bn_name)(x)
    if activation is not None:
        ac_name = _generate_layer_name('Activation', prefix=name)
        x = Activation(activation, name=ac_name)(x)
    return x


def _generate_layer_name(name, branch_idx=None, prefix=None):
    if prefix is None:
        return None
    if branch_idx is None:
        return '_'.join((prefix, name))
    return '_'.join((prefix, 'Branch', str(branch_idx), name))


def _inception_resnet_block(x, scale, block_type, block_idx, activation='relu'):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else 3
    if block_idx is None:
        prefix = None
    else:
        prefix = '_'.join((block_type, str(block_idx)))
    name_fmt = partial(_generate_layer_name, prefix=prefix)

    if block_type == 'Block35':
        branch_0 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 32, 3, name=name_fmt('Conv2d_0b_3x3', 1))
        branch_2 = conv2d_bn(x, 32, 1, name=name_fmt('Conv2d_0a_1x1', 2))
        branch_2 = conv2d_bn(branch_2, 32, 3, name=name_fmt('Conv2d_0b_3x3', 2))
        branch_2 = conv2d_bn(branch_2, 32, 3, name=name_fmt('Conv2d_0c_3x3', 2))
        branches = [branch_0, branch_1, branch_2]
    elif block_type == 'Block17':
        branch_0 = conv2d_bn(x, 128, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 128, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 128, [1, 7], name=name_fmt('Conv2d_0b_1x7', 1))
        branch_1 = conv2d_bn(branch_1, 128, [7, 1], name=name_fmt('Conv2d_0c_7x1', 1))
        branches = [branch_0, branch_1]
    elif block_type == 'Block8':
        branch_0 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_1x1', 0))
        branch_1 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_0a_1x1', 1))
        branch_1 = conv2d_bn(branch_1, 192, [1, 3], name=name_fmt('Conv2d_0b_1x3', 1))
        branch_1 = conv2d_bn(branch_1, 192, [3, 1], name=name_fmt('Conv2d_0c_3x1', 1))
        branches = [branch_0, branch_1]
    else:
        raise ValueError('Unknown Inception-ResNet block type. '
                         'Expects "Block35", "Block17" or "Block8", '
                         'but got: ' + str(block_type))

    mixed = Concatenate(axis=channel_axis, name=name_fmt('Concatenate'))(branches)
    up = conv2d_bn(mixed,
                   K.int_shape(x)[channel_axis],
                   1,
                   activation=None,
                   use_bias=True,
                   name=name_fmt('Conv2d_1x1'))
    up = Lambda(scaling,
                output_shape=K.int_shape(up)[1:],
                arguments={'scale': scale})(up)
    x = add([x, up])
    if activation is not None:
        x = Activation(activation, name=name_fmt('Activation'))(x)
    return x


def InceptionResNetV1(input_shape=(160, 160, 3),
                      classes=128,
                      dropout_keep_prob=0.8,
                      weights_path=None):
    inputs = Input(shape=input_shape)
    x = conv2d_bn(inputs, 32, 3, strides=2, padding='valid', name='Conv2d_1a_3x3')
    x = conv2d_bn(x, 32, 3, padding='valid', name='Conv2d_2a_3x3')
    x = conv2d_bn(x, 64, 3, name='Conv2d_2b_3x3')
    x = MaxPooling2D(3, strides=2, name='MaxPool_3a_3x3')(x)
    x = conv2d_bn(x, 80, 1, padding='valid', name='Conv2d_3b_1x1')
    x = conv2d_bn(x, 192, 3, padding='valid', name='Conv2d_4a_3x3')
    x = conv2d_bn(x, 256, 3, strides=2, padding='valid', name='Conv2d_4b_3x3')

    # 5x Block35 (Inception-ResNet-A block):
    for block_idx in range(1, 6):
        x = _inception_resnet_block(x,
                                    scale=0.17,
                                    block_type='Block35',
                                    block_idx=block_idx)

    # Mixed 6a (Reduction-A block):
    channel_axis = 1 if K.image_data_format() == 'channels_first' else 3
    name_fmt = partial(_generate_layer_name, prefix='Mixed_6a')
    branch_0 = conv2d_bn(x,
                         384,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 0))
    branch_1 = conv2d_bn(x, 192, 1, name=name_fmt('Conv2d_0a_1x1', 1))
    branch_1 = conv2d_bn(branch_1, 192, 3, name=name_fmt('Conv2d_0b_3x3', 1))
    branch_1 = conv2d_bn(branch_1,
                         256,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 1))
    branch_pool = MaxPooling2D(3,
                               strides=2,
                               padding='valid',
                               name=name_fmt('MaxPool_1a_3x3', 2))(x)
    branches = [branch_0, branch_1, branch_pool]
    x = Concatenate(axis=channel_axis, name='Mixed_6a')(branches)

    # 10x Block17 (Inception-ResNet-B block):
    for block_idx in range(1, 11):
        x = _inception_resnet_block(x,
                                    scale=0.1,
                                    block_type='Block17',
                                    block_idx=block_idx)

    # Mixed 7a (Reduction-B block): 8 x 8 x 2080
    name_fmt = partial(_generate_layer_name, prefix='Mixed_7a')
    branch_0 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 0))
    branch_0 = conv2d_bn(branch_0,
                         384,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 0))
    branch_1 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 1))
    branch_1 = conv2d_bn(branch_1,
                         256,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 1))
    branch_2 = conv2d_bn(x, 256, 1, name=name_fmt('Conv2d_0a_1x1', 2))
    branch_2 = conv2d_bn(branch_2, 256, 3, name=name_fmt('Conv2d_0b_3x3', 2))
    branch_2 = conv2d_bn(branch_2,
                         256,
                         3,
                         strides=2,
                         padding='valid',
                         name=name_fmt('Conv2d_1a_3x3', 2))
    branch_pool = MaxPooling2D(3,
                               strides=2,
                               padding='valid',
                               name=name_fmt('MaxPool_1a_3x3', 3))(x)
    branches = [branch_0, branch_1, branch_2, branch_pool]
    x = Concatenate(axis=channel_axis, name='Mixed_7a')(branches)

    # 5x Block8 (Inception-ResNet-C block):
    for block_idx in range(1, 6):
        x = _inception_resnet_block(x,
                                    scale=0.2,
                                    block_type='Block8',
                                    block_idx=block_idx)
    x = _inception_resnet_block(x,
                                scale=1.,
                                activation=None,
                                block_type='Block8',
                                block_idx=6)

    # Classification block
    x = GlobalAveragePooling2D(name='AvgPool')(x)
    x = Dropout(1.0 - dropout_keep_prob, name='Dropout')(x)
    # Bottleneck
    x = Dense(classes, use_bias=False, name='Bottleneck')(x)
    bn_name = _generate_layer_name('BatchNorm', prefix='Bottleneck')
    x = BatchNormalization(momentum=0.995, epsilon=0.001, scale=False,
                           name=bn_name)(x)

    # Create model
    model = Model(inputs, x, name='inception_resnet_v1')
    if weights_path is not None:
        model.load_weights(weights_path)

    return model


facenet = InceptionResNetV1(weights_path='/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/facenet_keras_weights_VGGFace2.h5', classes=512)
#for getting 512 dimensional embedding vector facenet is there
facenet = Model(facenet.inputs, facenet.layers[-1].output)

In [58]:
from numpy import expand_dims
from sklearn.preprocessing import Normalizer

class Verification():
  def __init__(self,model):
    self.model = model

  def get_embedding(self, face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = expand_dims(face_pixels, axis=0)
    yhat = self.model.predict(samples)
    return self.normalize([yhat[0]])

  def normalize(self, data):
    in_encoder = Normalizer(norm='l2')
    data = in_encoder.transform(data)
    return (data)

for_face_verification = Verification(facenet)

In [59]:
from scipy.spatial import distance
class Distance():
  def __init__(self,embedding,names):
    self.embedding = embedding
    self.names = names

  def cosine_dist(self,emb1,emb2):
    return distance.cosine(emb1, emb2)

  def predict_name(self,pred):
    #initializing to -infinity
    min = float('inf')
    for index,grd_embedding in enumerate(self.embedding):
      dist = self.cosine_dist(grd_embedding,pred)
      if dist < min:
        min = dist  
        name = self.names[index]
    
    if (name == 'SHELDON' and min > 0.5) or (name == 'SHELDON' and min > 0.5) or (name == 'AMY' and min > 0.5) or (name == 'EMILY' and min > 0.5):
      name = 'UNKNOWN'
      return name
    if min < 0.675:
      return name
    else:
      return 'UNKNOWN'


In [60]:
#Class for detection frame by frame

class Detection():
  def __init__(self,embedding,image_names):
    self.names = []
    self.bboxes = None 
    self.embedding = embedding
    self.image_names = image_names
  
  def return_frame(self,frame):
    #This func will return the final image which have bounding box and name of the detected person
    for idx, item in enumerate(self.bboxes):
      p1 = (int(item[1]), int(item[0]))
      p2 = (int(item[3]), int(item[2]))
      
      text=self.names[idx]

      if text == 'UNKNOWN':
        #Bounding box and name('unknown') will appear in red color
        output = cv2.rectangle(frame,p1, p2, (255,25, 0), thickness=3)
        output = cv2.putText(output,text,(p1[0],p1[1]-10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255,0,0), 2)
      else:
        #bounding box and name will appear in green color
        output = cv2.rectangle(frame,p1, p2, (0,255, 0), thickness=3)
        output = cv2.putText(output,text,(p1[0],p1[1]-10), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,255,0), 2)

    return output

  def detection_verification(self,frame):
    #This func will detect bboxes
    self.bboxes = fd.detect_face(frame, with_landmarks = False)
    
    for image in self.bboxes:
      x0, y0, x1, y1, score = image # show the first detected face
      x0, y0, x1, y1 = map(int, [x0, y0, x1, y1])
      im = np.array(frame[x0:x1,y0:y1,:])
      im = cv2.resize(im,(160,160),cv2.INTER_AREA)
      item = np.array(im) 
      pred = for_face_verification.get_embedding(item)
      dist = Distance(self.embedding,self.image_names)
      self.names.append(dist.predict_name(pred))
    
    output = self.return_frame(frame)
    return output


In [61]:
from google.colab.patches import cv2_imshow

class inference():
  def __init__(self,embedding,names):
    self.embedding = embedding
    self.names = names

  def load_models(self,verifier, detector):
    self.verifier = verifier
    self.detector = detector 

  def read_video(self,path):
    self.cap = cv2.VideoCapture(path)
    fourcc_codec = cv2.VideoWriter_fourcc(*'XVID')
    fps = 20.0
    capture = (int(self.cap.get(3)), int(self.cap.get(4)))
    self.videowriter = cv2.VideoWriter('/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/output_video.avi', fourcc_codec, fps, capture)

  def run_inference(self):
    count = 0
    
    print('inferencing is started your video will be saved as output_video.avi')
    while(self.cap.isOpened()):
      # Capture frame-by-frame
      ret, frame0 = self.cap.read()
      if ret == True:
        count+=1
        try:
          frame = cv2.cvtColor(frame0,cv2.COLOR_BGR2RGB)
          detection = Detection(self.embedding, self.names)
          output = detection.detection_verification(frame)
        
          self.videowriter.write(output[:,:,::-1]) 
            
        except Exception as e:
          print(e)
          self.videowriter.write(frame[:,:,::-1])
          continue
      else:
        break
            
    # releasing_camera
    self.cap.release()
    # release videowriter
    self.videowriter.release()
    #destroying_window
    cv2.destroyAllWindows()

In [62]:
import cv2
import matplotlib.pyplot as plt
from numpy import expand_dims
from sklearn.preprocessing import Normalizer

def resize_image(im, max_size=768):
  if np.max(im.shape) > max_size:
      ratio = max_size / np.max(im.shape)
      #print(f"Resize image to ({str(int(im.shape[1]*ratio))}, {str(int(im.shape[0]*ratio))}).")
      return cv2.resize(im, (0,0), fx=ratio, fy=ratio)
  return im

def prepare_database():
  
  print('Preparing data base......')

  raj_im = cv2.imread('/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/images_big_bang_theory/input_images/raj1.jpg')
  howard_im = cv2.imread('/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/images_big_bang_theory/input_images/howard.jpg')
  leonard_im = cv2.imread('/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/images_big_bang_theory/input_images/leonard.jpg')
  penny_im = cv2.imread('/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/images_big_bang_theory/input_images/penny1.jpg')
  sheldon_im = cv2.imread('/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/images_big_bang_theory/input_images/sheldon.jpg')
  amy_im =cv2.imread('/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/images_big_bang_theory/input_images/Amy.jpg')
  emily_im = cv2.imread('/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/images_big_bang_theory/input_images/Emily.jpg')
  benadette = cv2.imread('/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/images_big_bang_theory/input_images/bernadette.jpg')
  stuart_im = cv2.imread('/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/images_big_bang_theory/input_images/stuart.jpg')

  images = []

  images.append(leonard_im)
  images.append(howard_im)
  images.append(penny_im)
  images.append(sheldon_im)
  images.append(raj_im)
  images.append(amy_im)
  images.append(emily_im)
  images.append(benadette)
  images.append(stuart_im)


  image_name = ['LEONARD', 'HOWARD', 'PENNY' , 'SHELDON', 'RAJ', 'AMY', 'EMILY', 'BERNADETTE', 'STUART']
  image_embedding = []

  for item in images:
    item = resize_image(np.array(item)[..., ::-1],max_size=768)
    item = cv2.cvtColor(item, cv2.COLOR_BGR2RGB)
    bboxes = fd.detect_face(item, with_landmarks=False)
    x0, y0, x1, y1, score = bboxes[0] # show the first detected face
    x0, y0, x1, y1 = map(int, [x0, y0, x1, y1])
    im = np.array(item[x0:x1,y0:y1,:])
    im = cv2.resize(im,(160,160),cv2.INTER_AREA)
    item = np.array(im) 
    image_embedding.append(for_face_verification.get_embedding(item))
  print('Database is ready.')
  return image_embedding,image_name


def main_inference(image_embedding,names):
  
  inf = inference(image_embedding,names)
  inf.read_video('/content/drive/MyDrive/projects_google_collab/Face_Detection_Verification/trial_video.mp4')
  inf.run_inference()

def main():
  image_embedding,names = prepare_database()
  main_inference(image_embedding,names)

In [63]:
main()

Preparing data base......


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local

Database is ready.
inferencing is started your video will be saved as output_video.avi


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local

OpenCV(4.1.2) /io/opencv/modules/imgproc/src/resize.cpp:3720: error: (-215:Assertion failed) !ssize.empty() in function 'resize'



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local